In [1]:
# ! pip install gmaps

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Import CSV
weather = pd.read_csv("../WeatherPy/Outputs/Weather_Data")
weather

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Saint-Joseph,RE,-21.3667,55.6167,23.24,61,75,5.66
1,Vaini,TO,-21.2000,-175.2000,30.00,79,75,4.12
2,Chokurdakh,RU,70.6333,147.9167,-27.66,79,73,1.91
3,Romanovskaya,RU,47.5426,42.0285,5.68,90,99,4.05
4,Longyearbyen,SJ,78.2186,15.6401,-18.00,71,40,6.17
...,...,...,...,...,...,...,...,...
577,Boromo,BF,11.7500,-2.9333,26.49,15,4,3.51
578,Barra de Tecoanapa,MX,16.5167,-98.7500,26.83,70,0,4.12
579,Tekeli,KZ,44.8300,78.8239,-0.82,92,100,3.31
580,Berbera,SO,10.4396,45.0143,22.87,74,74,2.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [18]:
# Store Latitude and Longitude into  locations and make float
locations = weather[["Latitude", "Longitude"]].astype(float)

#Ensure humidity is float typle
humid = weather["Humidity"].astype(float)

In [19]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center=(30, 0), zoom_level=2)

humid_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(humid_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Slim down df to locations with no cloudiness, wind < 10 mph, and temps between 21-24 degrees C (70-75 degree F)
#No cloudiness
no_clouds = weather.loc[weather['Cloudiness'] == 0]
no_clouds

#winds less than 10 mph
slow_wind = no_clouds.loc[no_clouds['Wind Speed'] < 10]
slow_wind

#temp between 18-26
ideal_weather = slow_wind.loc[(slow_wind['Temperature'] > 21) & (slow_wind['Temperature'] < 24)]
print(f'There are {len(ideal_weather)} cities in this dataset')
ideal_weather

There are 8 cities in this dataset


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
125,Viedma,AR,-40.8135,-62.9967,23.33,66,0,7.75
182,Teacapan,MX,22.5500,-105.7500,21.06,69,0,4.18
222,Port Elizabeth,ZA,-33.9180,25.5701,22.00,94,0,2.06
263,Sonoita,MX,31.8500,-112.8333,23.33,19,0,2.50
284,Tairua,NZ,-37.0167,175.8500,22.22,53,0,4.47
303,Tomatlán,MX,19.9333,-105.2500,23.19,59,0,4.31
460,Kaura Namoda,NG,12.5898,6.5779,22.82,18,0,4.89
490,Makkah al Mukarramah,SA,21.4267,39.8261,23.32,63,0,1.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame(ideal_weather, columns=['City', 'Country', 'Latitude', 'Longitude'])
hotel_df['Hotel Name']=""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
125,Viedma,AR,-40.8135,-62.9967,
182,Teacapan,MX,22.5500,-105.7500,
222,Port Elizabeth,ZA,-33.9180,25.5701,
263,Sonoita,MX,31.8500,-112.8333,
284,Tairua,NZ,-37.0167,175.8500,
303,Tomatlán,MX,19.9333,-105.2500,
460,Kaura Namoda,NG,12.5898,6.5779,
490,Makkah al Mukarramah,SA,21.4267,39.8261,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name_address = hotel_name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.") 

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
125,Viedma,AR,-40.8135,-62.9967,Platinum Apart
182,Teacapan,MX,22.5500,-105.7500,Hotel G&B Continental
222,Port Elizabeth,ZA,-33.9180,25.5701,The Kelway Hotel
263,Sonoita,MX,31.8500,-112.8333,Motel La Hacienda
284,Tairua,NZ,-37.0167,175.8500,Grand Mercure Puka Park Resort
303,Tomatlán,MX,19.9333,-105.2500,Las Alamandas
460,Kaura Namoda,NG,12.5898,6.5779,Green Scenery Hotel
490,Makkah al Mukarramah,SA,21.4267,39.8261,Pullman Zamzam Makkah


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))